In [1]:
pip install pyvi

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import re
import random
from pyvi import ViTokenizer
import numpy as np

In [3]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from os import walk
mypath='data'
f = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f.extend(filenames)
    break

In [5]:
label = [l.replace('.txt','') for l in f]
label

['shoping',
 'người yêu',
 'nghỉ lễ',
 'robot',
 'du lịch',
 'đất nước',
 'trò chuyện về đi ăn',
 'địa chỉ',
 'học tập',
 'tdtu',
 'tán gẫu',
 'nghề nghiệp',
 'thông tin cá nhân',
 'bạn bè',
 'sở thích',
 'các câu hỏi phức tạp',
 'giải trí',
 'gia đình']

In [6]:
for i in range(0,len(f)):
    linktxt='./data/'+f[i]
    if label[i]in linktxt:
      print(label[i])

shoping
người yêu
nghỉ lễ
robot
du lịch
đất nước
trò chuyện về đi ăn
địa chỉ
học tập
tdtu
tán gẫu
nghề nghiệp
thông tin cá nhân
bạn bè
sở thích
các câu hỏi phức tạp
giải trí
gia đình


In [7]:
questions =[]
answers=[]
labels=[]
for i in range(0,len(f)):
    linktxt='./data/'+f[i]
    with open(linktxt,encoding='UTF-8') as file:
        lines=file.readlines()
        for line in lines:
            tmp = line.split('__eou__')
            questions.append(tmp[0].strip())
            answers.append(tmp[1].strip())
            if label[i]in linktxt:
              labels.append(label[i])

In [8]:
df = pd.DataFrame(list(zip(questions,labels,answers)),columns=['Q','L','A'])
df.head(10)

,Q,L,A
0,"xin chào, mình có thể giúp gì cho bạn ?",shoping,mình cần mua một cái điện thoại mới
1,bạn muốn một chiếc điện thoại như thế nào ?,shoping,"mình chưa biết, bạn có thể tư vấn mình không ?"
2,ở đây có chiếc điện thoại nào kèm tai nghe khô...,shoping,những chiếc điện thoại có giá 6 triệu trở lên ...
3,điện thoại nào đang là thịnh hành hiện nay ?,shoping,hiện tại điện thoại Apple đang rất được ưa chuộng
4,bạn có tính mua sắm gì vào dịp tết này không?,shoping,mình tính sắm vài bộ quần áo và 1 đôi giày mới
5,bạn tính mua online hay offline?,shoping,mình tính mua offline
6,ứng dụng nào bạn hay dùng để mua sắm online?,shoping,mình hay sử dụng ứng dụng shopee
7,mua sắm bằng ứng dụng shopee có tiện không?,shoping,"rất tiện dụng, giá cả còn phải chăng nửa"
8,bạn có thể gợi ý cho mình chỗ nào mua laptop t...,shoping,bạn có thể ghé qua gearvn để mua laptop
9,mình tính mua 1 chiếc điện thoại mới bạn biết ...,shoping,bạn có thể đến mua ở cellphoneS


In [9]:
correct_mapping = {
    'lol':'lồn',
    'trog': 'trong',
    'ko': 'không',
    'hs': 'học sinh',
    'ne': 'nè',
    'chừ': 'giờ',
    'oto': 'ô tô',
    'mí': 'mới',
    'share': 'chia sẻ',
    'sgk': 'sách giáo khoa',
    'cv':'công việc',
    'kq':'kết quả',
    'mj': 'mày',
    'z':'vậy',
    'rụnng': 'rụng',
    'nhìu': 'nhiều',
    'ah': 'à',
    'k': 'không',
    'v': 'vậy',
    'àk': 'à',
    'ngulon': 'ngu lồn',
    'm': 'mày',
    'ks': 'khách sạn',
    'vkl':'vãi cả lồn',
    'dume': 'đụ mẹ',
    'fa': 'một mình',
    'nyc': ' người yêu cũ',
    'nge': 'nghe',
    'zi': 'vậy',
    't': 'tao',
    'boss': 'sếp',
    'of':'của',
    'immm': 'im',
    'volum':'âm lượng',
    'ghêeeee':'ghê',
    'crush':'người trong mộng',
    'gane':'trò chơi',
    'stt': 'trạng thái',
    'fải': 'phải',
    'vler':'vãi lồn',
    'pr':'quảng cáo',
    'tp': 'thành phố',
    'hcm': 'Hồ Chí Minh',
    'ơiii':'ơi',
    'q.tâm':'quan tâm',
    'xxx': 'phim khiêu dâm',
    'tr': 'trời',
    'nta':'người ta',
    'thậc': 'thật',
    'chớt': 'chết',
    'lsao' : 'làm sao',
    'auto': 'tự động',
    'luônnn': 'luôn',
    'gym': "thể hình",
    'tml': 'thằng mặt lồn',
    'bằg':'bằng',
    'klq':'không liên quan',
    'ẻ': 'ỉa',
    'disss':'địt',
    'đmaaaa':'đụ má',
    'yêuu':'yêu',
    "g": 'gì',
    'or': 'hoặc',
    'yêuc': 'yêu',
    'zo': 'vào',
    'chờiiiii': 'trời',
    'tôiii':'tôi',
    's':'sao',
    'r': 'rồi',
    'thôiiii':'thôi',
    'dth':"dễ thương",
    'vớiiiiii' :'với',
    'zoe': 'dơ',
    'hlv':'huấn luyện viên',
    'lozzz': 'lồn',
    'đm':'đụ má',
    'lònnnnnnn':'lồn',
    'xanhhhh':'xanh',
    'saolol':'xạo lồn',
    'chaii':'chai',
    'ak':'à',
    'pk': 'biết',
    'iêu': 'yêu',
    'thíu': 'thiếu',
    'caphe': 'cà phê',
    'ukm': 'ừ',
    'hp': 'hạnh phúc',
    'tau':'tao',
    'ac':'anh chị',
    'q.q':'quần què',
    'ddaau': 'đâu',
    'lào': 'nào',
    'facebook':'mạng xã hội',
    'vcllll':'vãi cả lồn',
    'taooo': 'tao',
    'taoooo': 'tao',
    'video': 'đoạn phim',
    'make': 'làm',
    'in':'tại',
    'copy': 'sao chép',
    'sayy': 'nói',
    'ng': 'người',
    'goodbye': 'tạm biệt',
    'bcs': 'bao cao su',
    'chowi': 'chơi',
    'dlm': 'đụ mẹ',
    'cmnl': 'con mẹ nó luôn',
    'cc': 'con cặc',
    'showbiz':'giới giải trí',
    'yep':'vâng',
    'vl':'vãi lồn',
    'vlonee':'vãi lồn',
    'vclll':'vãi cả lồn',
    'tg':'tưởng',
    'cungc': 'cũng',
    'đờii': 'đời',
    'hmmmmmmm':'',
    'bn': 'bạn',
    '1': 'một',
    '2': 'hai',
    '3': 'ba',
    '4': 'bốn',
    '5': 'năm',
    '6': 'sáu',
    '7': 'bảy',
    '8': 'tám',
    '9': 'chín',
    '0': 'không',
    'tphcm': 'thành phố hồ chí minh'
}

def check_acronyms(w):
    if w in correct_mapping:
        return correct_mapping[w]
    else:
        return w

In [10]:
def preprocessing(sent):
    sent=sent.lower()
    sent=re.findall(r'\b\S+\b',sent)
    sent = list(map(check_acronyms,sent))
    sent = [lemmatizer.lemmatize(w) for w in sent]
    sent= ' '.join(sent)
    new_sent=ViTokenizer.tokenize(sent)
    return new_sent

In [11]:
length_sent=[]
for i in range(len(df.Q)):
    df.Q[i]=preprocessing(df.Q[i])
df.head(10)

,Q,L,A
0,xin chào mình có_thể giúp gì cho bạn,shoping,mình cần mua một cái điện thoại mới
1,bạn muốn một chiếc điện_thoại như thế_nào,shoping,"mình chưa biết, bạn có thể tư vấn mình không ?"
2,ở đây có chiếc điện_thoại nào kèm tai_nghe không,shoping,những chiếc điện thoại có giá 6 triệu trở lên ...
3,điện_thoại nào đang là thịnh_hành hiện_nay,shoping,hiện tại điện thoại Apple đang rất được ưa chuộng
4,bạn có tính mua_sắm gì vào dịp tết này không,shoping,mình tính sắm vài bộ quần áo và 1 đôi giày mới
5,bạn tính mua online hay offline,shoping,mình tính mua offline
6,ứng_dụng nào bạn hay dùng để mua_sắm online,shoping,mình hay sử dụng ứng dụng shopee
7,mua_sắm bằng ứng_dụng shopee có tiện không,shoping,"rất tiện dụng, giá cả còn phải chăng nửa"
8,bạn có_thể gợi_ý cho mình chỗ nào mua laptop t...,shoping,bạn có thể ghé qua gearvn để mua laptop
9,mình tính mua một chiếc điện_thoại mới bạn biế...,shoping,bạn có thể đến mua ở cellphoneS


In [12]:
bag_of_word=[]
for i in df.Q:
    bag_of_word.extend(word_tokenize(i))
bag_of_word = sorted(list(set(bag_of_word)))

In [13]:
X=[]
for i in df.Q:
    bow=[]
    for w in bag_of_word:
        bow.append(1) if w in i else bow.append(0)
    X.append(bow)

In [14]:
y=[]
for i in df.L:
    emp=[0]*len(label)
    index = label.index(i)
    emp[index]=1
    y.append(emp)
y[:10]

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [16]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y[0]), activation='softmax'))
 
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
 
hist = model.fit(np.array(X), np.array(y), epochs=200, batch_size=18, verbose=1)


Epoch 1/200
326/326 [==============================] - 4s 7ms/step - loss: 2.5950 - accuracy: 0.2001
Epoch 2/200
326/326 [==============================] - 2s 6ms/step - loss: 2.1148 - accuracy: 0.3644
Epoch 3/200
326/326 [==============================] - 1s 4ms/step - loss: 1.8772 - accuracy: 0.4435
Epoch 4/200
326/326 [==============================] - 1s 4ms/step - loss: 1.7008 - accuracy: 0.4916
Epoch 5/200
326/326 [==============================] - 1s 4ms/step - loss: 1.6268 - accuracy: 0.5205
Epoch 6/200
326/326 [==============================] - 1s 4ms/step - loss: 1.5340 - accuracy: 0.5371
Epoch 7/200
326/326 [==============================] - 1s 4ms/step - loss: 1.4890 - accuracy: 0.5548
Epoch 8/200
326/326 [==============================] - 1s 4ms/step - loss: 1.4385 - accuracy: 0.5698
Epoch 9/200
326/326 [==============================] - 1s 4ms/step - loss: 1.3900 - accuracy: 0.5722
Epoch 10/200
326/326 [==============================] - 1s 4ms/step - loss: 1.3451 - accura

In [17]:
model.save('chatbot_model.h5', hist)

In [18]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
def predict_label(user_quest,model):
    res = model.predict(np.array(vec_quest))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": label[r[0]], "probability": str(r[1])})
    return return_list

In [84]:
from scipy.spatial.distance import cdist
def bot_answer(vec_quest,return_list,df):
    distance={}
    list_anwer = df.loc[df.L==return_list[0]['intent']].index.tolist()
    for j in list_anwer:
        distance[j]= cdist( [X[j]],vec_quest, 'euclidean')
    min_position= min(distance, key=distance.get)
    return df.A[min_position]

In [92]:
print('Mình có thể giúp gì được cho bạn')
while(True):
    user_question =  input()
    if user_question=="end":
        break
    user_question = preprocessing(user_question)
    vec_quest=[]
    bow=[]
    for w in bag_of_word:
        bow.append(1) if w in user_question else bow.append(0)
    vec_quest.append(bow)
    return_list = predict_label(vec_quest,model)
    distance=bot_answer(vec_quest,return_list,df)
    print(distance)
    print('***********//*********')
    


Mình có thể giúp gì được cho bạn
bạn có thể bao tui một ly trà sữa được không
tui không có tiền nên không bao được nha
***********//*********
solo không
Chấp lun 2 mạng đầu
***********//*********
đánh nhau không
Thôi, mất công dọn bàn nữa
***********//*********
bạn tên gì
Mình tên Phạm Thị Lan Anh.
***********//*********
trà sữa không
bao thì uống à hổng từ chối đâu
***********//*********
end
